In [1]:
import pandas as pd

df = pd.read_csv("../Data/home_depot_data_1_2021_12.csv")
df.head()

,url,title,images,description,product_id,sku,gtin13,brand,price,currency,availability,uniq_id,scraped_at
0,https://www.homedepot.com/p/Carhartt-Men-s-3X-...,Men's 3X Large Carbon Heather Cotton/Polyester...,https://images.thdstatic.com/productImages/a7c...,"This heavyweight, water-repellent hooded sweat...",310090686,1.004455e+09,8.868595e+11,Carhartt,64.99,USD,InStock,ba6b9373-a145-5203-a7fe-5d9ede759dbb,2021-12-14 00:55:53.060268
1,https://www.homedepot.com/p/Turmode-30-ft-RP-T...,Turmode 30 ft. RP TNC Female to RP TNC Male Ad...,https://images.thdstatic.com/productImages/87a...,If you need more length between your existing ...,206724580,1.001661e+09,6.697163e+11,Unbranded,71.61,USD,InStock,70b7dd77-6474-5085-9afa-cda486a7237d,2021-12-14 00:55:53.070474
2,https://www.homedepot.com/p/Carolina-Pet-Compa...,Large Tapestry Bolster Bed,https://images.thdstatic.com/productImages/e6b...,Polyester cover resembling rich Italian tapest...,310347105,1.005202e+09,8.300001e+11,Carolina Pet Company,166.83,USD,InStock,8f573cae-947d-5262-8e51-b9381e8babeb,2021-12-14 00:55:53.677894
3,https://www.homedepot.com/p/16-Gauge-Sinks-Ves...,16-Gauge-Sinks Vessel Sink in White with Faucet,https://images.thdstatic.com/productImages/a6e...,It features a rectangle shape. This vessel set...,312338711,1.005103e+09,6.279872e+11,Unbranded,507.63,USD,InStock,3867081f-ea97-5855-8d6d-603b8e81f983,2021-12-14 00:55:53.987718
4,https://www.homedepot.com/p/Adtec-Men-s-Crazy-...,Men's Crazy Horse 9'' Logger Boot - Steel Toe ...,https://images.thdstatic.com/productImages/e7f...,This 9 in. black full grain leather logger boo...,308561619,1.004096e+09,6.477880e+11,Adtec,103.59,USD,InStock,e810513f-e5ed-598b-a17b-7ab60a1170fb,2021-12-14 00:55:54.003789


In [24]:
from sqlalchemy import create_engine
import configparser

config = configparser.ConfigParser()
config.read("../Config/db.config")

username = config['PG']['POSTGRES_USER']
pwd = config['PG']['POSTGRES_PASSWORD']
dbname = config['PG']['POSTGRES_DB']
url = config['PG']['POSTGRES_URL']
port = config['PG']['POSTGRES_PORT']

engine = create_engine("postgresql+psycopg2://{}:{}@{}:{}/{}".format(username,pwd,url,port,dbname))

In [ ]:
df.to_sql(name='Products', con=engine)

In [ ]:
Products = df.to_dict('records')

In [33]:
import requests
import json

url = "http://127.0.0.1:15001/embedtext"
headers = {
  'Content-Type': 'application/json'
}


for i,product in enumerate(Products):
    payload = json.dumps({
        "id": i,
        "text": product['title']
    })
    response = requests.request("POST", url, headers=headers, data=payload)
    product['embedding'] = response.json()['embedding']
    

In [39]:
from pymongo import MongoClient

username = config['MONGODB']['MONGO_USER']
pwd = config['MONGODB']['MONGO_PASSWORD']
dbname = config['MONGODB']['MONGO_DB']
url = config['MONGODB']['MONGO_URL']
port = config['MONGODB']['MONGO_PORT']

client = MongoClient(url,
                     username=username,
                     password=pwd,
                     authSource="admin",
                     authMechanism='SCRAM-SHA-256')

In [40]:
db = client[dbname]
collection = db['Product']
collection.insert_many(Products)

InsertManyResult([ObjectId('65a42f54480523773174d83a'), ObjectId('65a42f54480523773174d83b'), ObjectId('65a42f54480523773174d83c'), ObjectId('65a42f54480523773174d83d'), ObjectId('65a42f54480523773174d83e'), ObjectId('65a42f54480523773174d83f'), ObjectId('65a42f54480523773174d840'), ObjectId('65a42f54480523773174d841'), ObjectId('65a42f54480523773174d842'), ObjectId('65a42f54480523773174d843'), ObjectId('65a42f54480523773174d844'), ObjectId('65a42f54480523773174d845'), ObjectId('65a42f54480523773174d846'), ObjectId('65a42f54480523773174d847'), ObjectId('65a42f54480523773174d848'), ObjectId('65a42f54480523773174d849'), ObjectId('65a42f54480523773174d84a'), ObjectId('65a42f54480523773174d84b'), ObjectId('65a42f54480523773174d84c'), ObjectId('65a42f54480523773174d84d'), ObjectId('65a42f54480523773174d84e'), ObjectId('65a42f54480523773174d84f'), ObjectId('65a42f54480523773174d850'), ObjectId('65a42f54480523773174d851'), ObjectId('65a42f54480523773174d852'), ObjectId('65a42f54480523773174d8